# Part 3: Deep Learning & Neural Networks

## Introduction

Neural networks mimic the human brain to make predictions by identifying underlying relationships in data. Deep learning consists of machine learning techniques that use neural networks to learn complex patterns from data. Keras is a beginner-friendly library for training neural networks using TensorFlow. 

## Neural Networks

Neural networks are computational models inspired by the human brain, composed of layers of nodes that represent neurons. They mimic the process by which neurons process input information to produce an output using an avtivation function. Adjusting weights minimizes prediction error and maximizes learning power. These consist of input layers that recieve information, hidden layers that perform calculations, output layers that produce the final prediction, weights that set learning parameters, and activation functions and control non-linearity. Advantages are high flexibility, integration with complex non-linear relationships, and ability to work with structured and unstructured data. Disadvantages are the large amount of data required, difficult interpretability, sensitivity to overfitting, and that it is computationally expensive. Use cases include language processing, and image recognition.

### Case Study

In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# neural network class
class NeuralNetwork():
    
    def __init__(self, input_size):
        np.random.seed(1)
        # Now uses dynamic input size
        self.synaptic_weights = 2 * np.random.random((input_size, 1)) - 1
    
    #def sigmoid(self, x):
        #return 1 / (1 + np.exp(-x))

    def sigmoid(self, x):
        x = np.clip(x, -500, 500)
        return 1 / (1 + np.exp(-x))

    
    def sigmoid_derivative(self, x):
        return x / (1 + x)  # or use x * (1 - x) for standard
    
    def train(self, training_inputs, training_outputs, training_iterations):
        for itr in range(training_iterations):
            output = self.think(training_inputs)
            error = training_outputs - output
            adjustments = np.dot(training_inputs.T, error * self.sigmoid_derivative(output))
            self.synaptic_weights += adjustments
    
    def think(self, inputs):
        inputs = inputs.astype(float)
        return self.sigmoid(np.dot(inputs, self.synaptic_weights))


In [10]:
# load datasets
data_path = "C:/Users/scgoo/OneDrive/Documents/colleges/Jacksonville/Spring 2025/Math 470 - ML Algorithms/githubMath470/Final Project/data/"
train_df = pd.read_csv(data_path + "mnist_train.csv")
test_df = pd.read_csv(data_path + "mnist_test.csv")
X_train = train_df.drop(columns=["label"])
y_train = train_df["label"].values.reshape(-1, 1)
X_test = test_df.drop(columns=["label"], errors='ignore')  
y_test = test_df["label"] if "label" in test_df.columns else None

# encode categorical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_cat = to_categorical(y_encoded)

# standardize 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

C:\Users\scgoo\anaconda3\envs\ml-env\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
# initialize neural network from class
nn = NeuralNetwork(input_size=X_train_scaled.shape[1])

In [24]:
# train
nn.train(X_train_scaled, y_train, training_iterations=100)

In [25]:
# predict
predictions = nn.think(X_test_scaled)
predicted_classes = (predictions > 0.5).astype(int)

# output
print("Predictions:")
print(predicted_classes.flatten())

Predictions:
[1 0 0 ... 1 0 0]


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Predict on test set
predictions = nn.think(X_test_scaled)
predicted_classes = (predictions > 0.5).astype(int)

# If test labels are available
y_test = test_df["label"].values.reshape(-1, 1)

# Metrics
accuracy = accuracy_score(y_test, predicted_classes)
precision = precision_score(y_test, predicted_classes, average='weighted')
recall = recall_score(y_test, predicted_classes, average='weighted')
f1 = f1_score(y_test, predicted_classes, average='weighted')

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Accuracy:  0.1250
Precision: 0.0247
Recall:    0.1250
F1 Score:  0.0412


C:\Users\scgoo\anaconda3\envs\ml-env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Report

The [MNIST](https://www.kaggle.com/datasets/oddrationale/mnist-in-csv) data used for this model is from kaggle. This neural network predicts labels based on pixel values where there are 784 pixels. Accuracy of 0.1250 indicates 12.5% of the pridictions are correct. Precision and recall of 0.0247 and 0.1250, respectively, indicate there are many more false positives than false negatives. The f1 score of 0.0412 further highlights the imbalance in false positives and negatives. Overall, this model did not perform with strong predictive power but could be improved with further tuning and/or additional iterations.

## Deep Learning with Keras

Keras is a high-level API that works with TensorFlow to simplify building, training, and deploying deep learning models. TensorFlow is an open source platform by Google used for building and deploying machine learning models. Keras works by defining a model, stacking layers, compiling with model.compile(), training with model.fit(), and evaluating with model.evaluate(). Advantages are ease of use, easy integration, built-in GPA support, and high customizability. Disadvantages are the lower-level control that is limited, contraints in flexibility for extremely complex models, and the fundamental knowledge requirement for effective use. Use cases are text generation and autoencoders. 

### Case Study

In [55]:
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Paths
train_dir = data_path + "keras/train/train"
test_dir = data_path + "keras/test/test"
labels_csv = data_path + "keras/trainLabels.csv"

# Read labels
labels_df = pd.read_csv(labels_csv)

# change filenames to strings
labels_df["label"] = labels_df["label"].astype(str)
labels_df["id"] = labels_df["id"].astype(str) + ".png"

# for reading images
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=train_dir,
    x_col="id",
    y_col="label",
    target_size=(64, 64),
    class_mode="categorical",
    subset="training",
    batch_size=32,
    shuffle=True
)

val_generator = datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=test_dir,
    x_col="id",
    y_col="label",
    target_size=(64, 64),
    class_mode="categorical",
    subset="validation",
    batch_size=32,
    shuffle=False
)

# Get number of classes
num_classes = len(train_generator.class_indices)

# Build model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Found 40000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


In [65]:
import os
import tensorflow as tf
import pandas as pd

val_dir = data_path + "keras/test/test"

# Load labels 
labels_df = pd.read_csv(data_path + "keras/trainLabels.csv")
labels_df["id"] = labels_df["id"].astype(str) + ".png"
label_to_index = {label: idx for idx, label in enumerate(sorted(labels_df["label"].unique()))}
labels_df["label_int"] = labels_df["label"].map(label_to_index)

# Build lists of file paths and labels from validation data
filepaths = [os.path.join(val_dir, fname) for fname in labels_df["id"]]
labels = labels_df["label_int"].values

# function to load and preprocess the images
def process_image(filepath, label):
    # Read the file
    image = tf.io.read_file(filepath)
    # Decode the PNG image. tf.image.decode_image can be used for PNG or JPEG.
    image = tf.image.decode_png(image, channels=3)
    # Resize image to 64x64 pixels
    image = tf.image.resize(image, [64, 64])
    # Normalize the image to [0, 1]
    image = image / 255.0
    return image, label

# Create the dataset
val_dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
val_dataset = val_dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [69]:
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer

# Load test labels
val_df = labels_df.sample(frac=0.2, random_state=42)  

# Load images manually
images = []
labels = []

for _, row in val_df.iterrows():
    img_path = os.path.join(train_dir, row["id"]) 
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (64, 64))
        img = img / 255.0  
        images.append(img)
        labels.append(row["label"])

X_val = np.array(images)
y_val = np.array(labels)

# Encode labels to one-hot
lb = LabelBinarizer()
y_val_encoded = lb.fit_transform(y_val)

# Predict
y_pred_probs = model.predict(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_val_encoded, axis=1)

# compute metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))


313/313 [==============================] - 3s 7ms/step
Accuracy: 0.0894
Precision: 0.059098928862100664
Recall: 0.0903860976019655
F1 Score: 0.047628329795483294


C:\Users\scgoo\anaconda3\envs\ml-env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Report 

The [CIFAR-10](https://www.kaggle.com/c/cifar-10) data used for this model is from kaggle. This deep learning model with karas performs object recognition with computer vision. The dataset include 60000 32x32 color images and associated labels stored in a csv file. These labels make evaluation possible. The accuracy of 0.894 indicates that 8.94% of the preditions are correct. Precision and recall of 0.0591 and 0.0904, respectively, indicate there are more false positives than false negatives. The f1 score of 0.0476 speaks to the overall problem with false predictions. Tuning and more iterations are required to determine if this method has potential for strong predicitve power with this dataset. 